In [1]:
!uname -a

Linux m6.lobos.nih.gov 3.10.0-1160.80.1.el7.x86_64 #1 SMP Tue Nov 8 15:48:59 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
import apocharmm as ch

ImportError: libchcudadynlib.so: cannot open shared object file: No such file or directory

In [3]:
prm = ch.CharmmParameters(["../test/data/par_all36_prot.prm", "../test/data/par_all36_lipid.prm", "../test/data/toppar_water_ions.str"])
psf = ch.CharmmPSF("../test/data/walp.psf")
crd = ch.CharmmCrd("../test/data/walp.crd")

In [4]:
fm = ch.ForceManager(psf, prm)
fm.setBoxDimensions([53.4630707, 53.4630707, 80.4928487])
fm.setFFTGrid(48, 48, 48)
fm.setKappa(0.34)
fm.setCutoff(9.0)
fm.setCtonnb(7.0)
fm.setCtofnb(8.0)
fm.initialize()

In [5]:
ctx = ch.CharmmContext(fm)
ctx.setCoordinates(crd)
ctx.assignVelocitiesAtTemperature(300)

In [6]:
integrator = ch.VelocityVerletIntegrator(0.001)
integrator.setSimulationContext(ctx)

subscriber = ch.DcdSubscriber("out/vv_walp.dcd", ctx)
ctx.subscribe(subscriber)
#stateSub = ch.StateSubscriber("vv_walp.txt", ctx)
#ctx.subscribe(stateSub)

#subscriber = ch.NetCDFSubscriber("vv_walp.nc", ctx)
#ctx.subscribe(subscriber)
integrator.setReportSteps(10000)

#integrator.propagate(1000000)

In [7]:
ctx.getVolume()

230072.7042578221

In [8]:
53.4630707 * 53.4630707 * 80.4928487

230072.70369791257

In [9]:
ch.__file__

'/u/samar/.conda/envs/py39/lib/python3.9/site-packages/apocharmm-0.0.1-py3.9-linux-x86_64.egg/apocharmm.cpython-39-x86_64-linux-gnu.so'

In [10]:
ctx.calculatePressure()

1.0

In [11]:
integrator.propagate(2000)

# Zack's systems

In [12]:
import apocharmm as ch
psf = ch.CharmmPSF("/u/zjarin/for_samar/fullsystem.psf")
prm = ch.CharmmParameters(["../test/data/par_all36_prot.prm", "../test/data/par_all36_lipid.prm", "../test/data/toppar_water_ions.str"])

In [13]:
fm = ch.ForceManager(psf, prm)

In [14]:
#crd = ch.CharmmCrd("/u/zjarin/for_samar/edited_fullsystem.crd")
pdb = ch.PDB("/u/zjarin/for_samar/dyn500.pdb")

In [22]:
fm.setBoxDimensions([147.247,147.247,119.502])
fm.setFFTGrid(148, 148, 148)
fm.setKappa(0.34)
fm.setCutoff(12.0-3)
fm.setCtonnb(10.0-3)
fm.setCtofnb(11.0-3)
fm.initialize()

In [23]:
ctx = ch.CharmmContext(fm)
#ctx.setCoordinates(crd)
ctx.setCoordinates(pdb)
ctx.assignVelocitiesAtTemperature(300)

In [24]:
ctx.calculatePotentialEnergy(True, True)

-287199.1875

In [25]:
integrator = ch.VelocityVerletIntegrator(0.001)
integrator.setSimulationContext(ctx)


In [26]:
subscriber = ch.DcdSubscriber("out/vv_zack.dcd", ctx)
ctx.subscribe(subscriber)
#stateSub = ch.StateSubscriber("vv_walp.txt", ctx)
#ctx.subscribe(stateSub)

#subscriber = ch.NetCDFSubscriber("vv_walp.nc", ctx)
#ctx.subscribe(subscriber)
integrator.setReportSteps(10000)

In [20]:
integrator.propagate(2000)

In [27]:
%timeit -n 1 integrator.propagate(10000)

44.1 s ± 38.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
pdb.getNumAtoms()

267322

# Setting up a bilayer simulation

In [2]:
from charmm import apocharmm as ch

In [3]:
prm = ch.CharmmParameters(["../test/data/par_all36_prot.prm", "../test/data/par_all36_lipid.prm", 
                           "../test/data/toppar_all36_lipid_cholesterol.str","../test/data/toppar_water_ions.str"])
psf = ch.CharmmPSF("../test/data/fp.psf")
crd = ch.CharmmCrd("../test/data/min_p21.crd")

In [4]:
fm = ch.ForceManager(psf, prm)
fm.setBoxDimensions([64.52, 64.52, 102.02])
fm.setFFTGrid(72, 72, 108)
fm.setKappa(0.34)
fm.setCutoff(12.5)
fm.setCtonnb(9.0)
fm.setCtofnb(11.0)
fm.setPeriodicBoundaryCondition(ch.P21)

In [6]:
ctx = ch.CharmmContext(fm)
ctx.setCoordinates(crd)
ctx.assignVelocitiesAtTemperature(298)

In [13]:
integrator = ch.LangevinThermostatIntegrator(0.001)
integrator.setFriction(5.0)
integrator.setBathTemperature(298.17)

integrator.setSimulationContext(ctx)

In [14]:
subscriber = ch.DcdSubscriber("out/p21_bilayer_nvt.dcd", 1000,  ctx)
integrator.subscribe(subscriber)

In [11]:
#numSteps = 100000000
#integrator.propagate(numSteps)